In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pytz
from matplotlib.ticker import FuncFormatter

task_time = "16:00"  

task_time = datetime.strptime(task_time, "%H:%M").time()

start_time = datetime.strptime("15:00", "%H:%M").time()
end_time   = datetime.strptime("17:00", "%H:%M").time()

df = pd.read_csv("Play Store Data.csv")
df.columns = df.columns.str.strip().str.replace(" ", "_")

df['Last_Updated'] = pd.to_datetime(df['Last_Updated'], errors='coerce')

def convert_size(size):
    if isinstance(size, str):
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'k' in size:
            return float(size.replace('k', '')) / 1024
    return None

df['Size_MB'] = df['Size'].apply(convert_size)

df['Installs'] = df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

df = df.dropna(subset=['Category','Rating','Reviews','Installs','Size_MB','Last_Updated'])

filtered_df = df[
    (df['Rating'] >= 4.0) &
    (df['Size_MB'] >= 10) &
    (df['Last_Updated'].dt.month == 1)
]

top_categories = (
    filtered_df
    .groupby('Category', as_index=False)
    .agg({
        'Installs': 'sum',
        'Rating': 'mean',
        'Reviews': 'sum'
    })
    .sort_values(by='Installs', ascending=False)
    .head(10)
)

if start_time <= task_time <= end_time:

    print("✅ Time condition satisfied. Displaying graph...")

    x = range(len(top_categories))
    width = 0.4

    fig, ax1 = plt.subplots(figsize=(12,6))
    ax2 = ax1.twinx()

    ax1.bar(
        x,
        top_categories['Rating'],
        width=width,
        label='Average Rating',
        color='steelblue'
    )
    ax1.set_ylabel("Average Rating")
    ax1.set_ylim(0, 5)

    ax2.bar(
        [i + width for i in x],
        top_categories['Reviews'],
        width=width,
        label='Total Reviews',
        color='darkorange'
    )
    ax2.set_ylabel("Total Reviews")
    ax2.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x/1_000_000:.1f}M'))

    ax1.set_xticks([i + width/2 for i in x])
    ax1.set_xticklabels(top_categories['Category'], rotation=45, ha='right')
    ax1.set_xlabel("App Category")

    plt.title("Top 10 App Categories — Rating vs Reviews (Jan Updates)")

    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")

    plt.subplots_adjust(bottom=0.30)
    plt.show()

else:
    print("⏰ Chart will NOT be displayed (Allowed only between 3 PM and 5 PM IST).")